In [0]:
dbutils.widgets.text("proc_name", "", "Process Name")
dbutils.widgets.text("proc_run_id", "", "Process Run ID")
dbutils.widgets.text("data_dt", "", "Data Date")
proc_name = dbutils.widgets.get("proc_name")
proc_run_id = dbutils.widgets.get("proc_run_id")
data_dt = dbutils.widgets.get("data_dt")

In [0]:
%run /Workspace/Users/themallpocaws@inteltion.com/Data-Integration-Scenarios/00_common/common_function

In [0]:
%pip install paramiko
%pip install boto3 python-gnupg

In [0]:
import os
import gnupg
from pyspark.sql import SparkSession
import paramiko
from io import StringIO, BytesIO
import boto3
from botocore.exceptions import ClientError

In [0]:
def get_secret():
  secret_name = "TMG/POC/SFTP"
  region_name = "ap-southeast-1"

  # Create a Secrets Manager client
  session = boto3.session.Session()
  client = session.client(
      service_name='secretsmanager',
      region_name=region_name
  )

  try:
      get_secret_value_response = client.get_secret_value(
          SecretId=secret_name
      )
  except ClientError as e:
      # For a list of exceptions thrown, see
      # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
      raise e
  
  return json.loads(get_secret_value_response['SecretString'])

##### Encrypt file

In [0]:
batch_status=set_process_success()
try:
    # Read the table from Unity Catalog
    df = spark.table("poc_dwh.poc.poc_fact_sales")
    _data_dt=data_dt.replace("-","") 
    # Reduce to a single partition so that you get one CSV file
    df_single = df.coalesce(1)
    temp_output = 's3://tmg-poc-awsdb-apse1-stack-97db8-bucket/unity-catalog/catalog/poc_landing/temp_outbound/sftp_server/'
    # Write the DataFrame to S3 in CSV format with pipe delimiter and header
    df_single.write \
        .option("delimiter", ",") \
        .option("header", "true") \
        .mode("overwrite") \
        .csv(temp_output)

    # List the files in the temporary directory
    files = dbutils.fs.ls(temp_output)

    # Find the CSV file (the one that starts with "part-")
    csv_file = [f.path for f in files if f.name.endswith(".csv")][0]
    final_output = f"s3://tmg-poc-awsdb-apse1-stack-97db8-bucket/unity-catalog/catalog/poc_outbound/sftp_server/3_POC_Fact_Sales_{_data_dt}.csv"
    archived_output = f"s3://tmg-poc-awsdb-apse1-stack-97db8-bucket/unity-catalog/catalog/poc_outbound/sftp_server/archived/3_POC_Fact_Sales_{_data_dt}.csv"
    dbutils.fs.mv(csv_file, final_output)
except Exception as e:
    print(f"An error occurred: {e}")
    batch_status=set_process_failed(e)
    raise e

In [0]:
try:
    # Initialize Spark
    spark = SparkSession.builder.appName("EncryptCSVWithExistingPublicKey").getOrCreate()

    # Define S3 Paths
    tmg_public_key_s3 = 's3://tmg-poc-awsdb-apse1-stack-97db8-bucket/unity-catalog/catalog/poc_landing/TMG-POC-DWH_0x2CC43F58_public.asc'
    encrypted_csv_s3 = final_output + '.gpg'

    # Define Local Paths
    local_file_path = f'/tmp/output_{_data_dt}'
    local_pubkey_path = f"/tmp/TMG-POC-DWH_0x2CC43F58_public_{_data_dt}.asc"
    encrypted_csv_path = f"/tmp/output_{_data_dt}.csv.gpg"

    dbutils.fs.cp(final_output, "file:" + local_file_path)

    # Copy the existing public key from S3 to the local filesystem
    dbutils.fs.cp(tmg_public_key_s3, "file:" + local_pubkey_path)
    print(" Public key copied from S3 to:", local_pubkey_path)

    # Initialize GPG and import the public key
    gpg = gnupg.GPG()
    with open(local_pubkey_path, "r") as key_file:
        public_key = key_file.read()

    import_result = gpg.import_keys(public_key)
    if not import_result or not import_result.fingerprints:
        raise Exception("Public key import failed!")
    print("Public key imported. Fingerprint(s):", import_result.fingerprints)

    # PGP Encrypt
    with open(local_file_path, 'rb') as csv_file:
        encrypted_data = gpg.encrypt(csv_file.read(), recipients=['Siripatak.sir@themall.co.th'], always_trust=True)

    if not encrypted_data.ok:
        raise Exception("Encryption failed: " + encrypted_data.status)

    # Save the encrypted file locally
    with open(encrypted_csv_path, 'wb') as encrypted_file:
        encrypted_file.write(str(encrypted_data).encode())
    print("Encrypted file saved locally at:", encrypted_csv_path)

    # Upload the encrypted file to S3
    dbutils.fs.cp("file:" + encrypted_csv_path, encrypted_csv_s3)
    print("Encrypted file uploaded to:", encrypted_csv_s3)
except Exception as e:
    print(f"An error occurred: {e}")
    batch_status=set_process_failed(e)
    raise e


#### Send to sFTP 

In [0]:
# SFTP connection details
secret = get_secret() #Get SFTP secrets
hostname = secret["hostname"]
username = secret["username"]
password = secret["password"]
port = int(secret["port"])
gpg_decrypt_passphrase = secret["gpg_decrypt_passphrase"]


In [0]:

# S3 paths for the private key and the encrypted file
s3_input_file = encrypted_csv_s3 

# Define remote SFTP destination path (adjust as needed)
remote_file_path = f'/shared/uat/DWH_POC/In/PD3/3_POC_Fact_Sales_{_data_dt}.csv.gpg'

# Define local file paths
local_input_file_path = f"/tmp/3_POC_Fact_Sales_{_data_dt}.csv.gpg"

try:
    # Copy the encrypted file from S3 to the local filesystem
    dbutils.fs.cp(s3_input_file, "file:" + local_input_file_path)
    print("File copied from S3 to local path:", local_input_file_path)


    # Create a transport object for the SFTP connection
    transport = paramiko.Transport((hostname, port))
    transport.connect(username=username, password=password)
    
    # Create an SFTP session
    sftp = paramiko.SFTPClient.from_transport(transport)
    
    # Upload the file from the local path to the SFTP server
    sftp.put(local_input_file_path, remote_file_path)
    print("File uploaded to SFTP server at:", remote_file_path)
    
    # Close the SFTP session and transport
    sftp.close()
    transport.close()
    archived_output_encypted = f"s3://tmg-poc-awsdb-apse1-stack-97db8-bucket/unity-catalog/catalog/poc_outbound/sftp_server/archived/3_POC_Fact_Sales_{_data_dt}.csv.gpg"
    dbutils.fs.mv(final_output, archived_output)
    dbutils.fs.mv(s3_input_file, archived_output_encypted)
except Exception as e:
    print(f"Failed to connect to SFTP server: {e}")
    batch_status=set_process_failed(e)

dbutils.notebook.exit(batch_status)
